In [67]:
import pandas as pd

In [68]:
movies = pd.read_csv('data/cleaned-2/movies.csv').drop(columns='Unnamed: 0')
metadata = pd.read_csv('data/cleaned-2/metadata.csv').drop(columns='Unnamed: 0')

In [69]:
from langchain.docstore.document import Document

In [70]:
metadata['soup'] = movies['soup']

In [71]:
movies = []

for idx, row in metadata.iterrows():
    x = Document(page_content=row['soup'],
                 metadata = {
                     "movie": row['title'],
                     "language": row['original_language'],
                     "popularity": row['popularity'],
                     "synopsis": row['overview'],
                     "poster_path": row['poster_path']
                 })
    movies.append(x)

In [72]:
movies[0]

Document(metadata={'movie': 'Ariel', 'language': 'fi', 'popularity': 11.403, 'synopsis': 'After the coal mine he works at closes and his father commits suicide, a Finnish man leaves for the city to make a living but there, he is framed and imprisoned for various crimes.', 'poster_path': '/ojDg0PGvs6R9xYFodRct2kdI6wC.jpg'}, page_content='Title: Ariel. Genres: Comedy Drama Romance Crime. Keywords: prison underdog helsinki finland factory worker prisoner falling in love. Cast: TuroPajala SusannaHaavisto MattiPellonpää EetuHilkamo ErkkiPajala MattiJaaranen HannuViholainen JormaMarkkula TarjaKeinänen EinoKuusela. Directors: AkiKaurismäki. Writers: AkiKaurismäki. Overview: After the coal mine he works at closes and his father commits suicide, a Finnish man leaves for the city to make a living but there, he is framed and imprisoned for various crimes.. Released in: 1988. Productions: VillealfaFilmproductions. Original Language: fi')

In [73]:
len(movies)

3000

In [74]:
import weaviate

In [75]:
WEAVIATE_API_KEY = "y6KjOx9MDHF0lCUOt7XSPL8p3ZPtiPeBFmdC"
client = weaviate.Client(
    url="https://cjscqn5qa6ddwpdf5ucna.c0.asia-southeast1.gcp.weaviate.cloud",
    auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY)
)

C:\Users\Parag\anaconda3\Lib\site-packages\weaviate\__init__.py:144: DeprecationWarning: Dep010: Importing AuthApiKey from weaviate is deprecated. Import AuthApiKey from its module: weaviate.auth
  _Warnings.root_module_import(name, map_[name])
C:\Users\Parag\AppData\Local\Temp\ipykernel_8688\2194345016.py:2: DeprecationWarning: 
Python client v3 `weaviate.Client(...)` connections and methods are deprecated and will
            be removed by 2024-11-30.

            Upgrade your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.
                - For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
                - For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration

            If you have to use v3 code, install the v3 client and pin the v3 dependency in your requirements file: `weaviate-client>=3.26.7;<4.0.0`
  client = weaviate.Client(
C:\Users\Parag\anacon

In [76]:
from langchain_huggingface import HuggingFaceEmbeddings

In [77]:
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
  model_name=embedding_model_name,
  model_kwargs=model_kwargs
)

In [78]:
from langchain.vectorstores import Weaviate

In [79]:
vector_db = Weaviate.from_documents(
    movies,
    embeddings,
    client = client,
    by_text = False
)

In [80]:
soups = pd.Series(metadata['soup'].values, index=metadata['title'])

In [81]:
soups

title
Ariel                                 Title: Ariel. Genres: Comedy Drama Romance Cri...
Shadows in Paradise                   Title: Shadows in Paradise. Genres: Comedy Dra...
Four Rooms                            Title: Four Rooms. Genres: Comedy. Keywords: h...
Judgment Night                        Title: Judgment Night. Genres: Action Crime Th...
Life in Loops (A Megacities RMX)      Title: Life in Loops (A Megacities RMX). Genre...
                                                            ...                        
What to Do in Case of Fire?           Title: What to Do in Case of Fire?. Genres: Th...
Peter Voss, Thief of Millions         Title: Peter Voss, Thief of Millions. Genres: ...
Love at Twenty                        Title: Love at Twenty. Genres: Drama Romance. ...
New World Disorder 9: Never Enough    Title: New World Disorder 9: Never Enough. Key...
Sesame Street: Elmo Loves You!        Title: Sesame Street: Elmo Loves You!. Genres:...
Length: 3000, dtype: objec

In [87]:
def get_recommendations(title):

    if title not in soups:
        raise ValueError(f"Title '{title}' not found in indices")

    query = soups.get(title)

    try:
        results = vector_db.similarity_search_with_score(query, k=11)

        top_ten = []

        for x in results[1:]:
            movie_metadata = {
                'movie': x[0].metadata['movie'],
                'language': x[0].metadata['language'],
                'popularity': x[0].metadata['popularity'],
                'synopsis': x[0].metadata['synopsis'],
                'similarity_score': round(x[1], 2),

            }
            top_ten.append(movie_metadata)

        df_top_ten = pd.DataFrame(top_ten)
        return df_top_ten

    except Exception as e:
        print(f"Error during query: {e}")
        return None

In [88]:
metadata.title

0                                    Ariel
1                      Shadows in Paradise
2                               Four Rooms
3                           Judgment Night
4         Life in Loops (A Megacities RMX)
                       ...                
2995           What to Do in Case of Fire?
2996         Peter Voss, Thief of Millions
2997                        Love at Twenty
2998    New World Disorder 9: Never Enough
2999        Sesame Street: Elmo Loves You!
Name: title, Length: 3000, dtype: object

In [92]:
querry = metadata[metadata['title'].str.contains('Pirate')]['title'].values[0]
get_recommendations(querry)

,movie,language,popularity,synopsis,similarity_score
0,Pirates of the Caribbean: On Stranger Tides,en,91.486,Captain Jack Sparrow crosses paths with a woma...,0.74
1,Pirates of the Caribbean: At World's End,en,90.361,Will Turner and Elizabeth Swann join forces wi...,0.74
2,Pirates of the Caribbean: Dead Man's Chest,en,101.473,Captain Jack Sparrow works his way out of a bl...,0.72
3,Cutthroat Island,en,13.394,"Morgan Adams and her slave, William Shaw, are ...",0.58
4,Poseidon,en,22.958,A packed cruise ship traveling the Atlantic is...,0.50
5,Speed 2: Cruise Control,en,16.481,A disgruntled former employee hijacks the Seab...,0.49
6,The Poseidon Adventure,en,17.533,"When their ocean liner capsizes, a group of pa...",0.48
7,The Lost Continent,en,5.821,An eclectic group of characters set sail on Ca...,0.48
8,The Bounty,en,14.614,"The familiar story of Lieutenant Bligh, whose...",0.47
9,Pleasurecraft,en,1.001,A crew of astronauts are tasked with deliverin...,0.45
